# Ajuste fino de SMLs com Hugging Face

Nesta postagem, veremos como ajustar modelos de linguagem pequenos, como ajustar a classificação de texto e a geração de texto. Primeiro, veremos como fazer isso com as bibliotecas da Hugging Face, já que a Hugging Face se tornou um participante muito importante no ecossistema de IA atualmente.

Mas, embora as bibliotecas do Hugging Face sejam muito importantes e úteis, é muito importante saber como o treinamento é realmente feito e o que está acontecendo por trás dele, portanto, vamos repetir o treinamento para classificação e geração de texto, mas com o Pytorch.

Este caderno foi traduzido automaticamente para torná-lo acessível a mais pessoas, por favor me avise se você vir algum erro de digitação..

## Ajuste fino para classificação de texto com o Hugging Face

### Login

Para fazer upload do resultado do treinamento no hub, primeiro precisamos fazer login e, para isso, precisamos de um token.

Para criar um token, acesse a página [setings/tokens](https://huggingface.co/settings/tokens) de sua conta, que terá a seguinte aparência

User-Access-Token-dark](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/User-Access-Token-dark.webp)

Clique em `New token` e será exibida uma janela para criar um novo token.

![new-token-dark](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/new-token-dark.webp)

Nomeamos o token e o criamos com a função `write` ou com a função `Fine-grained`, que nos permite selecionar exatamente quais permissões o token terá.

Depois de criado, copiamos e colamos o arquivo abaixo

In [100]:
from huggingface_hub import notebook_login
notebook_login()

### Conjunto de dados

Agora, faremos o download de um conjunto de dados. Neste caso, faremos o download de um conjunto de dados de avaliações da [Amazon](https://huggingface.co/datasets/mteb/amazon_reviews_multi)

In [1]:
from datasets import load_dataset

dataset = load_dataset("mteb/amazon_reviews_multi", "en")

Vamos dar uma olhada nisso

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 5000
    })
})

Vemos que você tem um conjunto de treinamento com 200.000 amostras, um conjunto de validação com 5.000 amostras e um conjunto de teste com 5.000 amostras.

Vamos dar uma olhada em um exemplo do conjunto de treinamento

In [3]:
from random import randint

idx = randint(0, len(dataset['train']) - 1)
dataset['train'][idx]

{'id': 'en_0907914',
 'text': 'Mixed with fir it’s passable\n\nNot the scent I had hoped for . Love the scent of cedar, but this one missed',
 'label': 3,
 'label_text': '3'}

Vemos que você tem a avaliação no campo `text` e a pontuação dada pelo usuário no campo `label`.

Como vamos criar um modelo de classificação de texto, precisamos saber quantas classes teremos.

In [4]:
num_classes = len(dataset['train'].unique('label'))
num_classes

5

Teremos 5 classes, agora vamos ver o valor mínimo dessas classes para saber se a pontuação começa em 0 ou 1. Para isso, usamos o método `unique`

In [5]:
dataset.unique('label')

{'train': [0, 1, 2, 3, 4],
 'validation': [0, 1, 2, 3, 4],
 'test': [0, 1, 2, 3, 4]}

O valor mínimo será 0

Para o treinamento, os rótulos precisam estar em um campo chamado `labels`, enquanto em nosso conjunto de dados eles estão em um campo chamado `label`, portanto, criamos o novo campo `lables` com o mesmo valor de `label`.

Criamos uma função que faz o que queremos

In [6]:
def set_labels(example):
    example['labels'] = example['label']
    return example

Aplicamos a função ao conjunto de dados

In [7]:
dataset = dataset.map(set_labels)

Vejamos como é o conjunto de dados

In [8]:
dataset['train'][idx]

{'id': 'en_0907914',
 'text': 'Mixed with fir it’s passable\n\nNot the scent I had hoped for . Love the scent of cedar, but this one missed',
 'label': 3,
 'label_text': '3',
 'labels': 3}

### Tokeniser

Como temos revisões de texto no conjunto de dados, precisamos tokenizá-las para colocar os tokens no modelo.

In [9]:
from transformers import AutoTokenizer

checkpoint = "openai-community/gpt2"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Agora criamos uma função para tokenizar o texto. Vamos fazer com que todas as frases tenham o mesmo comprimento, de modo que o tokenizador truncará quando necessário e adicionará tokens de preenchimento quando necessário. Também pedimos que ele retorne tensores pytorch.

O comprimento de cada frase é de 768 tokens porque estamos usando o modelo GPT2 pequeno, que, como vimos na postagem [GPT2](https://maximofn.com/gpt2/#Arquitectura), tem uma dimensão de incorporação de 768 tokens.

In [10]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=768, return_tensors="pt")

Vamos tentar tokenizar um texto

In [11]:
tokens = tokenize_function(dataset['train'][idx])

ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

Recebemos um erro porque o tokenizador GPT2 não tem um token para preenchimento e nos pede para atribuir um, sugerindo que façamos `tokenizer.pad_token = tokenizer.eos_token`, e assim o fazemos.

In [100]:
tokenizer.pad_token = tokenizer.eos_token

Testamos novamente a função de tokenização.

In [100]:
tokens = tokenize_function(dataset['train'][idx])
tokens['input_ids'].shape, tokens['attention_mask'].shape

(torch.Size([1, 768]), torch.Size([1, 768]))

Agora que verificamos que a função tokeniza bem, aplicamos essa função ao conjunto de dados, mas também a aplicamos em lotes para que seja executada mais rapidamente.

Também tiramos proveito disso e eliminamos as colunas que não são necessárias.

In [100]:
dataset = dataset.map(tokenize_function, batched=True, remove_columns=['text', 'label', 'id', 'label_text'])

Agora vamos ver como é o conjunto de dados

In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text', 'labels'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['id', 'text', 'label', 'label_text', 'labels'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text', 'labels'],
        num_rows: 5000
    })
})

Vemos que temos os campos "labels" (rótulos), "input_ids" (IDs de entrada) e "attention_mask" (máscara de atenção), que é o que nos interessa treinar.

### Modelo

Instanciamos um modelo para classificação de sequências e informamos a ele o número de classes que temos.

In [13]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=num_classes)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Ele nos informa que os pesos da camada `score` foram inicializados aleatoriamente e que precisamos treiná-los novamente.

O modelo GPT2 teria a seguinte aparência

In [14]:
from transformers import AutoModelForCausalLM

casual_model = AutoModelForCausalLM.from_pretrained(checkpoint)

Enquanto o modelo GPT2 para gerar texto é o seguinte

Vamos dar uma olhada em sua arquitetura

In [15]:
casual_model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

E agora a arquitetura do modelo que usaremos para classificar as avaliações.

In [16]:
model

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=5, bias=False)
)

Há dois aspectos a serem mencionados aqui

 + A primeira é que, em ambas, a primeira camada tem dimensões de 50257x768, o que corresponde a 50257 tokens possíveis do vocabulário GPT2 e 768 dimensões da incorporação, portanto, fizemos bem em tokenizar as avaliações com um tamanho de 768 tokens.
 + A segunda é que o modelo `casual` (o modelo de geração de texto) tem, no final, uma camada `Linear` que gera 50257 valores, ou seja, é responsável por prever o próximo token e atribui um valor a cada token possível. Já o modelo de classificação tem uma camada `Linear` que gera apenas 5 valores, um para cada classe, o que nos dará a probabilidade de a avaliação pertencer a cada classe.

É por isso que recebemos a mensagem de que os pesos da camada `score` foram inicializados aleatoriamente, porque a biblioteca de transformadores removeu a camada `Linear` de 768x50257 e adicionou uma camada `Linear` de 768x5, inicializou-a com valores aleatórios e precisamos treiná-la para nosso problema específico.

Excluímos o modelo casual, porque não vamos usá-lo.

In [17]:
del casual_model

### Treinador

Vamos agora configurar os argumentos de treinamento

In [18]:
from transformers import TrainingArguments

metric_name = "accuracy"
model_name = "GPT2-small-finetuned-amazon-reviews-en-classification"
LR = 2e-5
BS_TRAIN = 28
BS_EVAL = 40
EPOCHS = 3
WEIGHT_DECAY = 0.01

training_args = TrainingArguments(
    model_name,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LR,
    per_device_train_batch_size=BS_TRAIN,
    per_device_eval_batch_size=BS_EVAL,
    num_train_epochs=EPOCHS,
    weight_decay=WEIGHT_DECAY,
    lr_scheduler_type="cosine",
    warmup_ratio = 0.1,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

Definir uma métrica para o carregador de dados de validação

In [19]:
import numpy as np
from evaluate import load

metric = load("accuracy")

def compute_metrics(eval_pred):
    print(eval_pred)
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

Agora definimos o treinador

In [20]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Treinamos

In [21]:
trainer.train()

  0%|          | 0/600000 [00:00<?, ?it/s]

ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided ['label', 'labels']

Recebemos um erro novamente porque o modelo não foi atribuído a um token de preenchimento, portanto, assim como no tokenizador, nós o atribuímos ao modelo

In [25]:
model.config.pad_token_id = model.config.eos_token_id

Recriamos os argumentos do instrutor com o novo modelo, que agora tem um token de preenchimento, o instrutor e treinamos novamente.

In [100]:
training_args = TrainingArguments(
    model_name,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LR,
    per_device_train_batch_size=BS_TRAIN,
    per_device_eval_batch_size=BS_EVAL,
    num_train_epochs=EPOCHS,
    weight_decay=WEIGHT_DECAY,
    lr_scheduler_type="cosine",
    warmup_ratio = 0.1,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
    logging_dir="./runs",
)

trainer = Trainer(
    model,
    training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Agora que vimos que tudo está bem, podemos treinar.

In [28]:
trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy
1,0.807400,0.820341,0.652000
2,0.751900,0.802189,0.654600
3,0.718100,0.810221,0.657800


TrainOutput(global_step=21429, training_loss=0.7846888848762739, metrics={'train_runtime': 26367.7801, 'train_samples_per_second': 22.755, 'train_steps_per_second': 0.813, 'total_flos': 2.35173445632e+17, 'train_loss': 0.7846888848762739, 'epoch': 3.0})

### Avaliação

Depois de treinados, avaliamos o conjunto de dados de teste

In [29]:
trainer.evaluate(eval_dataset=dataset['test'])

{'eval_loss': 0.7973636984825134,
 'eval_accuracy': 0.6626,
 'eval_runtime': 76.3016,
 'eval_samples_per_second': 65.529,
 'eval_steps_per_second': 1.638,
 'epoch': 3.0}

### Publicar o modelo

Agora que temos nosso modelo treinado, podemos compartilhá-lo com o mundo, portanto, primeiro criamos um cartão de modelo.

In [100]:
trainer.create_model_card()

E agora podemos publicá-lo. Como a primeira coisa que fizemos foi fazer login no hub da huggingface, poderemos fazer o upload para o nosso hub sem nenhum problema.

In [100]:
trainer.push_to_hub()

### Uso do modelo

Limpamos o máximo possível

In [100]:
import torch
import gc


def clear_hardwares():
    torch.clear_autocast_cache()
    torch.cuda.ipc_collect()
    torch.cuda.empty_cache()
    gc.collect()


clear_hardwares()
clear_hardwares()

Como fizemos o upload do modelo em nosso hub, podemos baixá-lo e usá-lo.

In [5]:
from transformers import pipeline

user = "maximofn"
checkpoints = f"{user}/{model_name}"
task = "text-classification"
classifier = pipeline(task, model=checkpoints, tokenizer=checkpoints)

Agora, se quisermos retornar a probabilidade de todas as classes, basta usar o classificador que acabamos de instanciar, com o parâmetro `top_k=None`.

In [8]:
labels = classifier("I love this product", top_k=None)
labels

[{'label': 'LABEL_4', 'score': 0.8253807425498962},
 {'label': 'LABEL_3', 'score': 0.15411493182182312},
 {'label': 'LABEL_2', 'score': 0.013907806016504765},
 {'label': 'LABEL_0', 'score': 0.003939222544431686},
 {'label': 'LABEL_1', 'score': 0.0026572425849735737}]

Se quisermos apenas a classe com a maior probabilidade, faremos o mesmo, mas com o parâmetro `top_k=1`.

In [9]:
label = classifier("I love this product", top_k=1)
label

[{'label': 'LABEL_4', 'score': 0.8253807425498962}]

E se quisermos n classes, faremos o mesmo, mas com o parâmetro `top_k=n`.

In [10]:
two_labels = classifier("I love this product", top_k=2)
two_labels

[{'label': 'LABEL_4', 'score': 0.8253807425498962},
 {'label': 'LABEL_3', 'score': 0.15411493182182312}]

Também podemos testar o modelo com o Automodel e o AutoTokenizer.

In [18]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_name = "GPT2-small-finetuned-amazon-reviews-en-classification"
user = "maximofn"
checkpoint = f"{user}/{model_name}"
num_classes = 5

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=num_classes).half().eval().to("cuda")

In [26]:
tokens = tokenizer.encode("I love this product", return_tensors="pt").to(model.device)
with torch.no_grad():
    output = model(tokens)
logits = output.logits
lables = torch.softmax(logits, dim=1).cpu().numpy().tolist()
lables[0]

[0.003963470458984375,
 0.0026721954345703125,
 0.01397705078125,
 0.154541015625,
 0.82470703125]

Se você quiser testar o modelo com mais detalhes, poderá vê-lo em [Maximofn/GPT2-small-finetuned-amazon-reviews-en-classification](https://huggingface.co/Maximofn/GPT2-small-finetuned-amazon-reviews-en-classification)

## Ajuste fino para geração de texto com o Hugging Face

Para ter certeza de que não tenho problemas de memória VRAM, reiniciei o notebook.

### Login

Para fazer upload do resultado do treinamento no hub, primeiro precisamos fazer login e, para isso, precisamos de um token.

Para criar um token, acesse a página [setings/tokens](https://huggingface.co/settings/tokens) de sua conta, que terá a seguinte aparência

User-Access-Token-dark](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/User-Access-Token-dark.webp)

Clique em `New token` e será exibida uma janela para criar um novo token.

![new-token-dark](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/new-token-dark.webp)

Nomeamos o token e o criamos com a função `write` ou com a função `Fine-grained`, que nos permite selecionar exatamente quais permissões o token terá.

Depois de criado, copiamos e colamos o arquivo abaixo

In [100]:
from huggingface_hub import notebook_login
notebook_login()

### Conjunto de dados

Vamos usar um conjunto de dados de [piadas em inglês] (https://huggingface.co/datasets/Maximofn/short-jokes-dataset)

In [1]:
from datasets import load_dataset

jokes = load_dataset("Maximofn/short-jokes-dataset")
jokes

DatasetDict({
    train: Dataset({
        features: ['ID', 'Joke'],
        num_rows: 231657
    })
})

Vamos dar uma olhada nisso

In [2]:
jokes

DatasetDict({
    train: Dataset({
        features: ['ID', 'Joke'],
        num_rows: 231657
    })
})

Vemos que se trata de um único conjunto de treinamento com mais de 200.000 piadas. Portanto, mais adiante, teremos de dividi-lo em treinamento e avaliação.

Vamos dar uma olhada em um exemplo

In [3]:
from random import randint

idx = randint(0, len(jokes['train']) - 1)
jokes['train'][idx]

{'ID': 198387,
 'Joke': 'My hot dislexic co-worker said she had an important massage to give me in her office... When I got there, she told me it can wait until I put on some clothes.'}

Vemos que ela tem uma identificação da piada que não nos interessa em absoluto e a piada em si

Caso você tenha pouca memória de GPU, farei um subconjunto do conjunto de dados, escolha a porcentagem de piadas que deseja usar.

In [4]:
percent_of_train_dataset = 1    # If you want 50% of the dataset, set this to 0.5

subset_dataset = jokes["train"].select(range(int(len(jokes["train"]) * percent_of_train_dataset)))
subset_dataset

Dataset({
    features: ['ID', 'Joke'],
    num_rows: 231657
})

Agora, dividimos o subconjunto em um conjunto de treinamento e um conjunto de validação.

In [5]:
percent_of_train_dataset = 0.90

split_dataset = subset_dataset.train_test_split(train_size=int(subset_dataset.num_rows * percent_of_train_dataset), seed=19, shuffle=False)
train_dataset = split_dataset["train"]
validation_test_dataset = split_dataset["test"]

split_dataset = validation_test_dataset.train_test_split(train_size=int(validation_test_dataset.num_rows * 0.5), seed=19, shuffle=False)
validation_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

print(f"Size of the train set: {len(train_dataset)}. Size of the validation set: {len(validation_dataset)}. Size of the test set: {len(test_dataset)}")

Size of the train set: 208491. Size of the validation set: 11583. Size of the test set: 11583


### Tokeniser

Instanciar o tokenizador. Instanciar o token de preenchimento do tokenizador para não recebermos um erro como antes.

In [6]:
from transformers import AutoTokenizer

checkpoints = "openai-community/gpt2"

tokenizer = AutoTokenizer.from_pretrained(checkpoints)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Para maior controle, adicionaremos dois novos tokens para o início e o fim de uma piada.

In [7]:
new_tokens = ['<SJ>', '<EJ>']   # Start and end of joke tokens

num_added_tokens = tokenizer.add_tokens(new_tokens)
print(f"Added {num_added_tokens} tokens")

Added 2 tokens


Criamos uma função para adicionar os novos tokens às sentenças

In [8]:
joke_column = "Joke"

def format_joke(example):
    example[joke_column] = '<SJ> ' + example['Joke'] + ' <EJ>'
    return example

Selecionamos as colunas de que não precisamos

In [9]:
remove_columns = [column for column in train_dataset.column_names if column != joke_column]
remove_columns

['ID']

Formatamos o conjunto de dados e excluímos as colunas de que não precisamos.

In [10]:
train_dataset = train_dataset.map(format_joke, remove_columns=remove_columns)
validation_dataset = validation_dataset.map(format_joke, remove_columns=remove_columns)
test_dataset = test_dataset.map(format_joke, remove_columns=remove_columns)
train_dataset, validation_dataset, test_dataset

(Dataset({
     features: ['Joke'],
     num_rows: 208491
 }),
 Dataset({
     features: ['Joke'],
     num_rows: 11583
 }),
 Dataset({
     features: ['Joke'],
     num_rows: 11583
 }))

Agora, criamos uma função para tokenizar as piadas.

In [11]:
def tokenize_function(examples):
    return tokenizer(examples[joke_column], padding="max_length", truncation=True, max_length=768, return_tensors="pt")

Tokenize o conjunto de dados e exclua a coluna com o texto

In [13]:
train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=[joke_column])
validation_dataset = validation_dataset.map(tokenize_function, batched=True, remove_columns=[joke_column])
test_dataset = test_dataset.map(tokenize_function, batched=True, remove_columns=[joke_column])
train_dataset, validation_dataset, test_dataset

(Dataset({
     features: ['input_ids', 'attention_mask'],
     num_rows: 208491
 }),
 Dataset({
     features: ['input_ids', 'attention_mask'],
     num_rows: 11583
 }),
 Dataset({
     features: ['input_ids', 'attention_mask'],
     num_rows: 11583
 }))

### Modelo

Agora, instanciamos o modelo para geração de texto e atribuímos o token de fim de cadeia ao token de carregamento.

In [14]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(checkpoints)
model.config.pad_token_id = model.config.eos_token_id

Vemos o tamanho do vocabulário do modelo

In [15]:
vocab_size = model.config.vocab_size
vocab_size

50257

Ele tem 50257 tokens, que é o tamanho do vocabulário do GPT2. Mas como dissemos que criaríamos dois novos tokens com o início da piada e o fim da piada, nós os adicionamos ao modelo

In [16]:
model.resize_token_embeddings(len(tokenizer))

new_vocab_size = model.config.vocab_size
print(f"Old vocab size: {vocab_size}. New vocab size: {new_vocab_size}. Added {new_vocab_size - vocab_size} tokens")

Old vocab size: 50257. New vocab size: 50259. Added 2 tokens


Os dois novos tokens foram adicionados

### Treinamento

Definimos os parâmetros de treinamento

In [17]:
from transformers import TrainingArguments

metric_name = "accuracy"
model_name = "GPT2-small-finetuned-Maximofn-short-jokes-dataset-casualLM"
output_dir = f"./training_results"
LR = 2e-5
BS_TRAIN = 28
BS_EVAL = 32
EPOCHS = 3
WEIGHT_DECAY = 0.01
WARMUP_STEPS = 100

training_args = TrainingArguments(
    model_name,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LR,
    per_device_train_batch_size=BS_TRAIN,
    per_device_eval_batch_size=BS_EVAL,
    warmup_steps=WARMUP_STEPS,
    num_train_epochs=EPOCHS,
    weight_decay=WEIGHT_DECAY,
    lr_scheduler_type="cosine",
    warmup_ratio = 0.1,
    fp16=True,
    load_best_model_at_end=True,
    # metric_for_best_model=metric_name,
    push_to_hub=True,
)

Agora não usamos `metric_for_best_model`, depois de definir o treinador, explicamos o motivo

Definimos o treinador

In [18]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    # compute_metrics=compute_metrics,
)

Nesse caso, não passamos uma função `compute_metrics`; se ela não for passada, a `loss` será usada durante a avaliação para avaliar o modelo. É por isso que, ao definir os argumentos, não definimos `metric_for_best_model`, pois não usaremos uma métrica para avaliar o modelo, mas a `loss`.

Treinamos

In [19]:
trainer.train()

  0%|          | 0/625473 [00:00<?, ?it/s]

ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.

Como podemos ver, ele nos dá um erro, diz que o modelo não retorna o valor da perda, o que é fundamental para podermos treinar.

Primeiro, vamos ver como é um exemplo do conjunto de dados.

In [100]:
idx = randint(0, len(train_dataset) - 1)
sample = train_dataset[idx]
sample

{'input_ids': [50257,
  4162,
  750,
  262,
  18757,
  6451,
  2245,
  2491,
  30,
  4362,
  340,
  373,
  734,
  10032,
  13,
  220,
  50258,
  50256,
  50256,
  ...,
  50256,
  50256,
  50256],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  ...,
  0,
  0,
  0]}

Como podemos ver, temos um dicionário com `input_ids` e `attention_mask` e, se o passarmos para o modelo, obteremos o seguinte

In [100]:
import torch

output = model(
    input_ids=torch.Tensor(sample["input_ids"]).long().unsqueeze(0).to(model.device),
    attention_mask=torch.Tensor(sample["attention_mask"]).long().unsqueeze(0).to(model.device),
)
print(output.loss)

None


Como podemos ver, ele não retorna o valor da perda porque está aguardando um valor para `labels`, que não foi passado a ele. No exemplo anterior, em que fizemos o ajuste fino para a classificação de texto, dissemos que os rótulos tinham de ser passados para um campo no conjunto de dados chamado `labels`, mas, nesse caso, não temos esse campo no conjunto de dados.

Se agora atribuirmos os `lables` aos `input_ids` e observarmos novamente a perda

In [100]:
import torch

output = model(
    input_ids=torch.Tensor(sample["input_ids"]).long().unsqueeze(0).to(model.device),
    attention_mask=torch.Tensor(sample["attention_mask"]).long().unsqueeze(0).to(model.device),
    labels=torch.Tensor(sample["input_ids"]).long().unsqueeze(0).to(model.device)
)
print(output.loss)

tensor(102.1873, device='cuda:0', grad_fn=<NllLossBackward0>)


Agora temos uma "perda".

Portanto, temos duas opções: adicionar um campo `labels` ao conjunto de dados, com os valores de `input_ids` ou usar uma função da biblioteca `transformers` chamada `data_collator`; nesse caso, usaremos `DataCollatorForLanguageModeling`. Vamos dar uma olhada nisso

In [100]:
from transformers import DataCollatorForLanguageModeling

my_data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Passamos a amostra `sample` por meio desse `data_collator`.

In [100]:
collated_sample = my_data_collator([sample]).to(model.device)

Vemos como é a saída

In [100]:
for key, value in collated_sample.items():
    print(f"{key} ({value.shape}): {value}")

input_ids (torch.Size([1, 768])): tensor([[50257,  4162,   750,   262, 18757,  6451,  2245,  2491,    30,  4362,
           340,   373,   734, 10032,    13,   220, 50258, 50256, ..., 50256, 50256]],
       device='cuda:0')
attention_mask (torch.Size([1, 768])): tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ..., 0, 0]],
       device='cuda:0')
labels (torch.Size([1, 768])): tensor([[50257,  4162,   750,   262, 18757,  6451,  2245,  2491,    30,  4362,
           340,   373,   734, 10032,    13,   220, 50258,  -100,  ...,  -100,  -100]],
       device='cuda:0')


Como você pode ver, o `data_collator` criou um campo `labels` e atribuiu a ele os valores de `input_ids`. Os tokens que são mascarados receberam um valor de -100. Isso se deve ao fato de que, quando definimos o `data_collator`, passamos a ele o parâmetro `mlm=False`, o que significa que não estamos fazendo `Masked Language Modeling`, mas `Language Modeling`, portanto, ele não mascara nenhum token original.

Vamos ver se agora temos uma `perda` com esse `data_collator`.

In [100]:
output = model(**collated_sample)
output.loss

tensor(102.7181, device='cuda:0', grad_fn=<NllLossBackward0>)

Portanto, redefinimos o `trainer` com o `data_collator` e treinamos novamente.

In [100]:
from transformers import DataCollatorForLanguageModeling

trainer = Trainer(
    model,
    training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
)

In [100]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.386600,3.258979
2,3.259900,3.199673
3,3.212600,3.192009


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=22341, training_loss=3.505178199598342, metrics={'train_runtime': 9209.5353, 'train_samples_per_second': 67.916, 'train_steps_per_second': 2.426, 'total_flos': 2.45146666696704e+17, 'train_loss': 3.505178199598342, 'epoch': 3.0})

### Avaliação

Depois de treinado, avaliamos o modelo no conjunto de dados de teste.

In [100]:
trainer.evaluate(eval_dataset=test_dataset)

{'eval_loss': 3.201305866241455,
 'eval_runtime': 65.0033,
 'eval_samples_per_second': 178.191,
 'eval_steps_per_second': 5.569,
 'epoch': 3.0}

### Publicar o modelo

Criamos o cartão modelo

In [100]:
trainer.create_model_card()

Nós o publicamos

In [100]:
trainer.push_to_hub()

events.out.tfevents.1720875425.8de3af1b431d.6946.1:   0%|          | 0.00/364 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Maximofn/GPT2-small-finetuned-Maximofn-short-jokes-dataset-casualLM/commit/d107b3bb0e02076483238f9975697761015ec390', commit_message='End of training', commit_description='', oid='d107b3bb0e02076483238f9975697761015ec390', pr_url=None, pr_revision=None, pr_num=None)

### Uso do modelo

Limpamos o máximo possível

In [100]:
import torch
import gc


def clear_hardwares():
    torch.clear_autocast_cache()
    torch.cuda.ipc_collect()
    torch.cuda.empty_cache()
    gc.collect()


clear_hardwares()
clear_hardwares()

Baixamos o modelo e o tokenizador

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

user = "maximofn"
checkpoints = f"{user}/{model_name}"

tokenizer = AutoTokenizer.from_pretrained(checkpoints)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(checkpoints)
model.config.pad_token_id = model.config.eos_token_id

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Verificamos se o tokenizador e o modelo têm os dois tokens extras que adicionamos.

In [10]:
tokenizer_vocab = tokenizer.get_vocab()
model_vocab = model.config.vocab_size
print(f"tokenizer_vocab: {len(tokenizer_vocab)}. model_vocab: {model_vocab}")

tokenizer_vocab: 50259. model_vocab: 50259


Vemos que eles têm 50259 tokens, ou seja, os 50257 tokens do GPT2 mais os 2 que adicionamos.

Criamos uma função para gerar piadas

In [14]:
def generate_joke(prompt_text):
    text = f"<SJ> {prompt_text}"
    tokens = tokenizer(text, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(**tokens, max_new_tokens=256, eos_token_id=tokenizer.encode("<EJ>")[-1])
    return tokenizer.decode(output[0], skip_special_tokens=False)

Geramos uma piada

In [15]:
generate_joke("Why didn't the frog cross the road?")

Setting `pad_token_id` to `eos_token_id`:50258 for open-end generation.


"<SJ> Why didn't the frog cross the road? Because he was frog-in-the-face. <EJ>"

Se você quiser testar o modelo com mais detalhes, poderá vê-lo em [Maximofn/GPT2-small-finetuned-Maximofn-short-jokes-dataset-casualLM](https://huggingface.co/Maximofn/GPT2-small-finetuned-Maximofn-short-jokes-dataset-casualLM)

## Ajuste fino para classificação de texto com o Pytorch

Repetimos o treinamento com o Pytorch

Reinicie o notebook para ter certeza de que

### Conjunto de dados

Fizemos o download do mesmo conjunto de dados que usamos no treinamento com as bibliotecas Hugging Face.

In [1]:
from datasets import load_dataset

dataset = load_dataset("mteb/amazon_reviews_multi", "en")

Criamos uma variável com o número de classes

In [2]:
num_classes = len(dataset['train'].unique('label'))
num_classes

5

Antes, processávamos todo o conjunto de dados para criar um campo chamado `labels`, mas agora isso não é necessário porque vamos programar tudo nós mesmos, portanto, nos adaptamos à aparência do conjunto de dados.

### Tokeniser

Criamos o tokenizador. Atribuímos o token de preenchimento para não recebermos um erro como antes.

In [100]:
from transformers import AutoTokenizer

checkpoint = "openai-community/gpt2"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.pad_token = tokenizer.eos_token

Criamos uma função para tokenizar o conjunto de dados

In [100]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=768, return_tensors="pt")

Nós a tokenizamos. Removemos as colunas que não são necessárias, mas agora deixamos a coluna de texto.

In [100]:
dataset = dataset.map(tokenize_function, batched=True, remove_columns=['id', 'label_text'])

In [100]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 5000
    })
})

In [100]:
percentage = 1
subset_train = dataset['train'].select(range(int(len(dataset['train']) * percentage)))
percentage = 1
subset_validation = dataset['validation'].select(range(int(len(dataset['validation']) * percentage)))
subset_test = dataset['test'].select(range(int(len(dataset['test']) * percentage)))
print(f"len subset_train: {len(subset_train)}, len subset_validation: {len(subset_validation)}, len subset_test: {len(subset_test)}")

len subset_train: 200000, len subset_validation: 5000, len subset_test: 5000


### Modelo

Importamos os pesos e atribuímos o token de preenchimento

In [100]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=num_classes)
model.config.pad_token_id = model.config.eos_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Dispositivo

Criamos o dispositivo onde tudo será executado

In [100]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Passamos o modelo para o dispositivo e o passamos para o FP16 para que ele ocupe menos memória.

In [100]:
model.half().to(device)
print()

### Conjunto de dados Pytorch

Criar um conjunto de dados pytorch

In [100]:
from torch.utils.data import Dataset

class ReviewsDataset(Dataset):
    def __init__(self, huggingface_dataset):
        self.dataset = huggingface_dataset

    def __getitem__(self, idx):
        label = self.dataset[idx]['label']
        input_ids = torch.tensor(self.dataset[idx]['input_ids'])
        attention_mask = torch.tensor(self.dataset[idx]['attention_mask'])
        return input_ids, attention_mask, label

    def __len__(self):
        return len(self.dataset)

Instanciar os conjuntos de dados

In [100]:
train_dataset = ReviewsDataset(subset_train)
validatation_dataset = ReviewsDataset(subset_validation)
test_dataset = ReviewsDataset(subset_test)

Vamos dar uma olhada em um exemplo

In [100]:
input_ids, at_mask, label = train_dataset[0]
input_ids.shape, at_mask.shape, label

(torch.Size([768]), torch.Size([768]), 0)

### Carregador de dados Pytorch

Criamos agora um carregador de dados pytorch

In [100]:
from torch.utils.data import DataLoader

BS = 12

train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True)
validation_loader = DataLoader(validatation_dataset, batch_size=BS)
test_loader = DataLoader(test_dataset, batch_size=BS)

Vamos dar uma olhada em um exemplo

In [100]:
input_ids, at_mask, labels = next(iter(train_loader))
input_ids.shape, at_mask.shape, labels

(torch.Size([12, 768]),
 torch.Size([12, 768]),
 tensor([2, 1, 2, 0, 3, 3, 0, 4, 3, 3, 4, 2]))

Para verificar se está tudo certo, passamos a amostra para o modelo para ver se está tudo certo. Primeiro, passamos os tokens para o dispositivo

In [100]:
input_ids = input_ids.to(device)
at_mask = at_mask.to(device)
labels = labels.to(device)

Agora, nós os passamos para o modelo

In [100]:
output = model(input_ids=input_ids, attention_mask=at_mask, labels=labels)
output.keys()

odict_keys(['loss', 'logits', 'past_key_values'])

Como podemos ver, ele nos fornece a perda e os logits.

In [100]:
output['loss']

tensor(5.9414, device='cuda:0', dtype=torch.float16,
       grad_fn=<NllLossBackward0>)

In [100]:
output['logits']

tensor([[ 6.1953e+00, -1.2275e+00, -2.4824e+00,  5.8867e+00, -1.4734e+01],
        [ 5.4062e+00, -8.4570e-01, -2.3203e+00,  5.1055e+00, -1.1555e+01],
        [ 6.1641e+00, -9.3066e-01, -2.5664e+00,  6.0039e+00, -1.4570e+01],
        [ 5.2266e+00, -4.2358e-01, -2.0801e+00,  4.7461e+00, -1.1570e+01],
        [ 3.8184e+00, -2.3460e-03, -1.7666e+00,  3.4160e+00, -7.7969e+00],
        [ 4.1641e+00, -4.8169e-01, -1.6914e+00,  3.9941e+00, -8.7734e+00],
        [ 4.6758e+00, -3.0298e-01, -2.1641e+00,  4.1055e+00, -9.3359e+00],
        [ 4.1953e+00, -3.2471e-01, -2.1875e+00,  3.9375e+00, -8.3438e+00],
        [-1.1650e+00,  1.3564e+00, -6.2158e-01, -6.8115e-01,  4.8672e+00],
        [ 4.4961e+00, -8.7891e-02, -2.2793e+00,  4.2812e+00, -9.3359e+00],
        [ 4.9336e+00, -2.6627e-03, -2.1543e+00,  4.3711e+00, -1.0742e+01],
        [ 5.9727e+00, -4.3152e-02, -1.4551e+00,  4.3438e+00, -1.2117e+01]],
       device='cuda:0', dtype=torch.float16, grad_fn=<IndexBackward0>)

### Métrica

Vamos criar uma função para obter a métrica, que, neste caso, será a precisão

In [100]:
def predicted_labels(logits):
    percent = torch.softmax(logits, dim=1)
    predictions = torch.argmax(percent, dim=1)
    return predictions

In [100]:
def compute_accuracy(logits, labels):
    predictions = predicted_labels(logits)
    correct = (predictions == labels).float()
    return correct.mean()

Vamos ver como ele calcula bem

In [100]:
compute_accuracy(output['logits'], labels).item()

0.1666666716337204

### Otimizador

Como precisaremos de um otimizador, criamos um.

In [100]:
from transformers import AdamW

LR = 2e-5
optimizer = AdamW(model.parameters(), lr=LR)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### Treinamento

Criamos o loop de treinamento

In [100]:
from tqdm import tqdm

EPOCHS = 3

accuracy = 0

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    progresbar = tqdm(train_loader, total=len(train_loader), desc=f'Epoch {epoch + 1}')
    for input_ids, at_mask, labels in progresbar:
        input_ids = input_ids.to(device)
        at_mask = at_mask.to(device)
        label = labels.to(device)

        output = model(input_ids=input_ids, attention_mask=at_mask, labels=label)

        loss = output['loss']
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        progresbar.set_postfix({'train_loss': loss.item()})
    train_loss /= len(train_loader)
    progresbar.set_postfix({'train_loss': train_loss})

    model.eval()
    valid_loss = 0
    progresbar = tqdm(validation_loader, total=len(validation_loader), desc=f'Epoch {epoch + 1}')
    for input_ids, at_mask, labels in progresbar:
        input_ids = input_ids.to(device)
        at_mask = at_mask.to(device)
        labels = labels.to(device)

        output = model(input_ids=input_ids, attention_mask=at_mask, labels=labels)

        loss = output['loss']
        valid_loss += loss.item()

        step_accuracy = compute_accuracy(output['logits'], labels)
        accuracy += step_accuracy
        progresbar.set_postfix({'valid_loss': loss.item(), 'accuracy': step_accuracy.item()})

    valid_loss /= len(validation_loader)
    accuracy /= len(validation_loader)
    progresbar.set_postfix({'valid_loss': valid_loss, 'accuracy': accuracy})

Epoch 3: 100%|██████████| 417/417 [00:32<00:00, 12.86it/s, valid_loss=nan, accuracy=0]


### Uso do modelo

Vamos testar o modelo que treinamos

Primeiro, tokenizamos um texto

In [100]:
input_tokens = tokenize_function({"text": "I love this product. It is amazing."})
input_tokens['input_ids'].shape, input_tokens['attention_mask'].shape

(torch.Size([1, 768]), torch.Size([1, 768]))

Agora, passamos isso para o modelo

In [100]:
output = model(input_ids=input_tokens['input_ids'].to(device), attention_mask=input_tokens['attention_mask'].to(device))
output['logits']

tensor([[nan, nan, nan, nan, nan]], device='cuda:0', dtype=torch.float16,
       grad_fn=<IndexBackward0>)

Vemos as previsões desses logits

In [100]:
predicted = predicted_labels(output['logits'])
predicted

tensor([0], device='cuda:0')

## Ajuste fino para geração de texto com o Pytorch

Repetimos o treinamento com o Pytorch

Reinicie o notebook para ter certeza de que

### Conjunto de dados

Baixamos novamente o conjunto de dados de piadas

In [1]:
from datasets import load_dataset

jokes = load_dataset("Maximofn/short-jokes-dataset")
jokes

DatasetDict({
    train: Dataset({
        features: ['ID', 'Joke'],
        num_rows: 231657
    })
})

Crie um subconjunto caso você esteja com pouca memória

In [2]:
percent_of_train_dataset = 1    # If you want 50% of the dataset, set this to 0.5

subset_dataset = jokes["train"].select(range(int(len(jokes["train"]) * percent_of_train_dataset)))
subset_dataset

Dataset({
    features: ['ID', 'Joke'],
    num_rows: 231657
})

Dividimos o conjunto de dados em subconjuntos de treinamento, validação e teste.

In [3]:
percent_of_train_dataset = 0.90

split_dataset = subset_dataset.train_test_split(train_size=int(subset_dataset.num_rows * percent_of_train_dataset), seed=19, shuffle=False)
train_dataset = split_dataset["train"]
validation_test_dataset = split_dataset["test"]

split_dataset = validation_test_dataset.train_test_split(train_size=int(validation_test_dataset.num_rows * 0.5), seed=19, shuffle=False)
validation_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

print(f"Size of the train set: {len(train_dataset)}. Size of the validation set: {len(validation_dataset)}. Size of the test set: {len(test_dataset)}")

Size of the train set: 208491. Size of the validation set: 11583. Size of the test set: 11583


### Tokeniser

Iniciamos o tokenizador e atribuímos o token do final da string ao token de preenchimento.

In [4]:
from transformers import AutoTokenizer

checkpoints = "openai-community/gpt2"

tokenizer = AutoTokenizer.from_pretrained(checkpoints)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Adicionamos os tokens especiais para o início e o fim de uma piada.

In [5]:
new_tokens = ['<SJ>', '<EJ>']   # Start and end of joke tokens

num_added_tokens = tokenizer.add_tokens(new_tokens)
print(f"Added {num_added_tokens} tokens")

Added 2 tokens


Nós os adicionamos ao conjunto de dados

In [6]:
joke_column = "Joke"

def format_joke(example):
    example[joke_column] = '<SJ> ' + example['Joke'] + ' <EJ>'
    return example

remove_columns = [column for column in train_dataset.column_names if column != joke_column]

train_dataset = train_dataset.map(format_joke, remove_columns=remove_columns)
validation_dataset = validation_dataset.map(format_joke, remove_columns=remove_columns)
test_dataset = test_dataset.map(format_joke, remove_columns=remove_columns)
train_dataset, validation_dataset, test_dataset

(Dataset({
     features: ['Joke'],
     num_rows: 208491
 }),
 Dataset({
     features: ['Joke'],
     num_rows: 11583
 }),
 Dataset({
     features: ['Joke'],
     num_rows: 11583
 }))

Tokenizamos o conjunto de dados

In [7]:
def tokenize_function(examples):
    return tokenizer(examples[joke_column], padding="max_length", truncation=True, max_length=768, return_tensors="pt")

train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=[joke_column])
validation_dataset = validation_dataset.map(tokenize_function, batched=True, remove_columns=[joke_column])
test_dataset = test_dataset.map(tokenize_function, batched=True, remove_columns=[joke_column])
train_dataset, validation_dataset, test_dataset

(Dataset({
     features: ['input_ids', 'attention_mask'],
     num_rows: 208491
 }),
 Dataset({
     features: ['input_ids', 'attention_mask'],
     num_rows: 11583
 }),
 Dataset({
     features: ['input_ids', 'attention_mask'],
     num_rows: 11583
 }))

### Modelo

Instanciar o modelo, atribuir o token de preenchimento e adicionar os novos tokens de início e fim de piada.

In [8]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(checkpoints)
model.config.pad_token_id = model.config.eos_token_id
model.resize_token_embeddings(len(tokenizer))

Embedding(50259, 768)

### Dispositivo

Criamos o dispositivo e passamos o modelo para o dispositivo.

In [9]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.half().to(device)
print()

### Conjunto de dados Pytorch

Criar um conjunto de dados pytorch

In [10]:
from torch.utils.data import Dataset

class JokesDataset(Dataset):
    def __init__(self, huggingface_dataset):
        self.dataset = huggingface_dataset

    def __getitem__(self, idx):
        input_ids = torch.tensor(self.dataset[idx]['input_ids'])
        attention_mask = torch.tensor(self.dataset[idx]['attention_mask'])
        return input_ids, attention_mask

    def __len__(self):
        return len(self.dataset)

Instanciamos os conjuntos de dados de treinamento, validação e teste.

In [11]:
train_pytorch_dataset = JokesDataset(train_dataset)
validation_pytorch_dataset = JokesDataset(validation_dataset)
test_pytorch_dataset = JokesDataset(test_dataset)

Aqui está um exemplo

In [12]:
input_ids, attention_mask = train_pytorch_dataset[0]
input_ids.shape, attention_mask.shape

(torch.Size([768]), torch.Size([768]))

### Carregador de dados Pytorch

Criamos os carregadores de dados

In [13]:
from torch.utils.data import DataLoader

BS = 28

train_loader = DataLoader(train_pytorch_dataset, batch_size=BS, shuffle=True)
validation_loader = DataLoader(validation_pytorch_dataset, batch_size=BS)
test_loader = DataLoader(test_pytorch_dataset, batch_size=BS)

Vemos uma amostra

In [13]:
input_ids, attention_mask = next(iter(train_loader))
input_ids.shape, attention_mask.shape

(torch.Size([28, 768]), torch.Size([28, 768]))

Passamos isso para o modelo

In [15]:
output = model(input_ids.to(device), attention_mask=attention_mask.to(device))
output.keys()

odict_keys(['logits', 'past_key_values'])

Como podemos ver, não temos nenhum valor de `perda`, pois, como vimos, temos que passar os `input_ids` e os `labels`.

In [16]:
output = model(input_ids.to(device), attention_mask=attention_mask.to(device), labels=input_ids.to(device))
output.keys()

odict_keys(['loss', 'logits', 'past_key_values'])

Agora temos a "perda".

In [17]:
output['loss'].item()

80.5625

### Otimizador

Criamos um otimizador

In [100]:
from transformers import AdamW

LR = 2e-5
optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### Treinamento

Criamos o loop de treinamento

In [100]:
from tqdm import tqdm

EPOCHS = 3

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    progresbar = tqdm(train_loader, total=len(train_loader), desc=f'Epoch {epoch + 1}')
    for input_ids, at_mask in progresbar:
        input_ids = input_ids.to(device)
        at_mask = at_mask.to(device)

        output = model(input_ids=input_ids, attention_mask=at_mask, labels=input_ids)

        loss = output['loss']
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        progresbar.set_postfix({'train_loss': loss.item()})
    train_loss /= len(train_loader)
    progresbar.set_postfix({'train_loss': train_loss})

Epoch 3: 100%|██████████| 7447/7447 [51:07<00:00,  2.43it/s, train_loss=nan]


### Uso do modelo

Testamos o modelo

In [100]:
def generate_text(decoded_joke, max_new_tokens=100, stop_token='<EJ>', top_k=0, temperature=1.0):
    input_tokens = tokenize_function({'Joke': decoded_joke})
    output = model(input_tokens['input_ids'].to(device), attention_mask=input_tokens['attention_mask'].to(device))
    nex_token = torch.argmax(output['logits'][:, -1, :], dim=-1).item()
    nex_token_decoded = tokenizer.decode(nex_token)
    decoded_joke = decoded_joke + nex_token_decoded
    for _ in range(max_new_tokens):
        nex_token = torch.argmax(output['logits'][:, -1, :], dim=-1).item()
        nex_token_decoded = tokenizer.decode(nex_token)
        if nex_token_decoded == stop_token:
            break
        decoded_joke = decoded_joke + nex_token_decoded
        input_tokens = tokenize_function({'Joke': decoded_joke})
        output = model(input_tokens['input_ids'].to(device), attention_mask=input_tokens['attention_mask'].to(device))
    return decoded_joke

In [100]:
generated_text = generate_text("<SJ> Why didn't the frog cross the road")
generated_text

"<SJ> Why didn't the frog cross the road!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!"